# TP2: Support Vector Machines

*By Daniel Deutsch and Kevin Khul*

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Exercise 1

In [4]:
def load_breastcancer(filename):
    data = np.loadtxt(filename, delimiter=',')

    # The column 0 isn't important for us here
    y = data[:, 1]*2 - 1
    X = data[:, 2:]

    # Standardisation of the matrix
    X = X - np.mean(X, axis=0)
    X = X / np.std(X, axis=0)

    return X, y

In [5]:
X, y = load_breastcancer("./datasets/wdbc_M1_B0.data")

# Exercise 2.1

## Question 2
### 2.1

From the proposed initial problem, we have (considering c = 1)

<br>

\begin{aligned}
f(v, a, \xi) = \frac{1}{2} \sum_j v_j^2 + \sum_i \xi_i
\end{aligned}

<br>

Observing the above function, we see that it is a increasing with $\xi_i$, $\forall i$ and $v_j$, $\forall j$.

Looking at the constraints of the original problem, we see that $\forall i$, $\xi_i$ must be greater or equal to zero and also greater or equal to $1 - y_i(x_i^Tv + a)$.

Therefore, we define a new $\xi_min \in \mathbb{R}^n$ as 

<br>

\begin{aligned}
\xi_{min_i} = max(0, 1 - y_i(x_i^Tv + a))
\end{aligned}

<br>

It stores the minimum value of $\xi_i, \forall i$.

Therefore, upon the considerations of a increasing function made above

<br>

\begin{aligned}
f(v, a, \xi) \ge f(v, a, \xi_{min}).
\end{aligned}

<br>

And, therefore

<br>

\begin{aligned}
min_{v,a,\xi} f(v,a,\xi) \ge min_{v,a,\xi} f(v, a, \xi_{min}) = min_{v,a} f(v, a, \xi_{min})
\end{aligned}

<br>

Besides, if we consider a function $g(x,y)$ which accepts a minimum, $\forall y_0 \in Y$ we have

<br>

\begin{aligned}
min_{x\in X, y \in Y} g(x,y) \le min_{x \in X} g(x,y_0)
\end{aligned}

Analogally,

<br>

\begin{aligned}
min_{v,a,\xi} f(v,a,\xi) \le min_{v,a} f(v, a, \xi_{min})
\end{aligned}

<br>

Using both inequalities, we can finally write

<br>

\begin{aligned}
min_{v,a,\xi} f(v,a,\xi) = min_{v,a} f(v, a, \xi_{min})
\end{aligned}

<br>

Which proves the equivalence.

# Exercise 2.2

Let's plot the function and the straight lines in between both existing gradients.

In [ ]:
def h(z):
    return np.maximum(np.zeros(z.shape), 1-z)

z = np.arange(-10, 10, 0.01)

plt.plot(z, h(z), color='blue', linewidth=3.0)
for j in range(21):
    i = 1
    s = -j/20
    y = (z-1)*s
    plt.plot(z, y, color='green', linewidth=0.3)
    
plt.legend(["h(z)", "Possible tangent"])
plt.show()

We see that h(z) is derivable for all z different from 1. Therefore, if $z \neq$ 1

<br>

\begin{aligned}
    \partial h(z) = \frac{dh(z)}{dz}
\end{aligned}

For z = 1, we have (by looking at the above plot) that all the slopes are in between -1 and 0, therefore 


\begin{aligned}
    \partial h(z) = 
    \begin{cases}
        \{-1\} & if \ z \lt 1 \\
        [-1;0] & if \ z = 1 \\
        \{0\} & if \ z \gt 1 \\
    \end{cases}
\end{aligned}

# Exercise 2.3

We first rewrite $f(v,a)$:

<br>

\begin{aligned}
    f(v, a) = \frac{1}{2}\cdotp\sum_{j=1}^{m} v^2_j + c\cdotp\sum_{i=1}^{n}max(0,1-y_i(x_i^Tv+a))
\end{aligned}

<br>

Our goal is to find N and H separable such that we can write:

<br>

\begin{aligned}
    f(v, a) = N(v, a) + c\cdotp H(M(v, a))
\end{aligned}

$N(v,a)$ is relatively easy to find, as we can write : 

<br>

\begin{aligned}
    N(v, a) = \sum_{j=1}^{m} h_j $ with $ h_j(v) = \frac{1}{2}v_j^2
\end{aligned}

<br>

Noting that it is, by construction, a separable function.

Now, when searching for $H(v, a)$ and $M(v, a)$, we know that

<br>
\begin{aligned}
    H(v, a)\cdotp M(v, a) = \sum_{i=1}^{n}max(0, 1-y_i(x_i^Tv+a))
\end{aligned}
<br>

To build M, we take a diagonal matrix D with all the elements $y_i$

<br>

\begin{aligned}
    D = \begin{pmatrix}
    y_1 & ... & 0 \\
    ... & ... & ...\\
    0 & ... & y_n
    \end{pmatrix}
\end{aligned}

<br>

Then, we create another matrix L, composed by the observations $x_i^T$ and complemented with a column of ones.

<br>

\begin{aligned}
    L= \begin{pmatrix}
    x_1^T & 1 \\
    ... & ...\\
    x_n^T & 1
    \end{pmatrix}
\end{aligned}

<br>

Finally, we can write M as $M = D\cdotp L$, and

<br>

\begin{aligned}
M(v, a) = M \cdotp \begin{pmatrix} v\\a\end{pmatrix}
\end{aligned}

<br>

Also we write H as $H = \sum_{i=1}^{n}h_i $, where $ h_i(W) = max(0, 1-w_i)$  for vector a $W$. Note that H is, also by construction, separable and W is the product of $M$ and $\begin{pmatrix}v\\a\end{pmatrix}$, yielding $M(v, a)$.

# Exercise 2.4

In [ ]:
X = np.insert(X,X.shape[1],1,axis=1)

In [ ]:
# Function to return randomly the subgradient of h taken at a given point
def H_subgradient(z):
    return ((z<1)*(-1)+(np.random.rand(1)[0]-1)*(z==1))

# Defining function f from the matrices and transformations found above
def f(v, a): 
    v_sum = 1/2*sum([v[j]**2 for j in range(len(v))])
    
    # Matrix D
    D = np.diag(y)
    
    # Matrix M
    M = np.dot(X.transpose(), D)
    
    # Matrix M(v,a)
    M_v_a = np.dot(M.transpose(), np.concatenate((v, [a]), axis=0))
    
    # Finally computing the second term
    c_sum = sum([max(0,1-M_v_a[i]) for i in range(len(M_v_a))])
    
    # Adding with the first term and returning
    return v_sum + c_sum

# Finally computing the gradient of f(v,a)
def subgradient(v, a):
    # Compute the subgradient of the first part of the function
    N_subgrad = np.concatenate((v,[0]),axis=0)
    
    # Defining matrices like in previous function 
    D = np.diag(y)
    M = np.dot(D, X)
    M_v_a = np.dot(M,np.concatenate((v, [a]), axis=0))
    
    # Use previous defined function to find the subgradient of the second part of the function f
    H_subgrad = np.array([H_subgradient(z) for z in M_v_a])
    
    # Multiplying it by the matrix M (transpose)
    multiplied_H_subgrad = np.dot(M.T, H_subgrad)
    
    # Returning the final result for the subgradient of f(v,a)
    return (N_subgrad + multiplied_H_subgrad)


In [ ]:
v = np.ones(30)
f(v, 1)

In [ ]:
subgradient(v, 1)

# Exercise 2.5

In [ ]:
# The subgradient function
def subgradient_method(v0,a0,epsilon=1,itmax=10000):
    gk = subgradient(v0,a0)
    xk = np.zeros(31)
    itr = 0
    while(np.linalg.norm(gk) > epsilon and itr <= itmax): 
        xk -= 1/(itr+1)*gk
        gk = subgradient(xk[:-1],xk[-1])
        print(f"\rCurrent iteration: {itr+1}/{itmax}", end="")
        itr += 1
    return xk

In [ ]:
v0 = np.zeros(30)

In [ ]:
x_min = subgradient_method(v0,0)

In [ ]:
print(f"The subgradient method gives a mininumum of: {f(x_min[:-1], x_min[-1]):.2f}, given the initial conditions (v0, a0) = 0")

# Exercise 3.1

We have that:

<br>

\begin{aligned}
    E[f_I (v, a)] \quad & = \quad \sum_{i = 1}^n P(I = i) \ f_i (v, a) \\
    & = \quad \frac{1}{n} \sum_{i = 1}^n f_i (v, a) \\
    & = \quad \frac{1}{n} \sum_{i = 1}^n \left(c \ n \ max(0, \ 1-y_i(x_i^T v + a)) + \frac{1}{2} \sum_{j = 1}^m v_j^2 \right) \\
    & = \quad \frac{1}{n} \sum_{i = 1}^n (c \ n \ max(0, \ 1-y_i(x_i^T v + a))) + \frac{1}{n} \sum_{i = 1}^n \frac{1}{2} \sum_{j = 1}^m v_j^2 \\
    & = \quad \sum_{i = 1}^n (c \ max(0, \ 1-y_i(x_i^T v + a))) + \frac{1}{2} \sum_{j = 1}^m v_j^2 \\
    & = \quad f(v, a)
\end{aligned}

# Exercise 3.2

Let $M_i$ be the $i_th$ row of $M$, i.e. $M_i = y_i(x_i^T, 1)$. This way, we have:

<br>

\begin{aligned}
    M_i\begin{pmatrix} v \\ a \end{pmatrix} \quad = \quad y_i(x_i^Tv + a)
\end{aligned}

<br>

Therefore, we can write $f_i(v, a) = N(v, a) + c \ n \ h(M_i(v,a))$, which implies $\partial f_i(v, a) = (v, 0) + c \ n \ M_i^T \ \partial h(M_i(v, a))$. Thus, we have:

<br>

\begin{aligned}
    \partial f_i(v, a) =
    \begin{cases}
        (v, 0) - n \ c \ M_i^T & if \ M_i(v, a) \lt 1 \\
        (v, 0) + t \ n \ c \ M_i^t, \quad t \in [-1; 0] & if \ M_i(v, a) = 1 \\
        (v, 0) & if \ M_i(v, a) \gt 1
    \end{cases}
\end{aligned}

# Exercise 3.3

In [ ]:
def delta_h_M_i(va, i):
    return ((np.dot(M[i,:],va)>=1)- 1) * M[i,:]

def delta_f_i(va, i):
    return delta_N(va) + c * M.shape[0] * delta_h_M_i(va,i)

def stoch_grad_method(va0, N):
    n = M.shape[0]
    va_moy = np.zeros(va0.shape)
    gamma_sum = 0
    
    for i in range(N):
        I = np.random.randint(n)
        gamma = 0.001/np.sqrt(i+1)
        gamma_sum += gamma
        va_moy += va0 * gamma
        
        va0 = va0 - gamma * delta_f_i(va0,I)
    
    #return va0
    return va_moy/gamma_sum

def stochastic_subgradient_method(M, itmax=10000):
    pass

# Exercise 4.1

The Lagrangian function $L()$ associated to Problem (1) is:

# Exercise 4.2

We have $g(x, \phi) = - \frac{1}{2\rho} \phi^2 + \frac{\rho}{2} (max(0, \ x + \rho^{-1} \phi))^2$ with $\rho > 0$. Firstly, to find $\nabla_x g(x, \phi)$, we do:

<br>

\begin{aligned}
    \nabla_x g(x, \phi) \quad = \quad \frac{\partial g(x, \phi)}{\partial x} \quad & = \quad \frac{\partial \left(-\frac{1}{2\rho} \phi^2\right)}{\partial x} + \frac{\partial \left(\frac{\rho}{2} (max(0, \ x + \rho^{-1} \phi))^2\right)}{\partial x} \\
    & = \quad 0 + \frac{2 \rho}{2} (max(0, \ x + \rho^{-1} \phi)) \\
    & = \quad \rho \ max(0, \ x + \rho^{-1} \phi)
\end{aligned}

<br>

Now, to find $\nabla_\phi g(x, \phi)$:

<br>

\begin{aligned}
    \nabla_\phi g(x, \phi) \quad = \quad \frac{\partial g(x, \phi)}{\partial \phi} \quad & = \quad \frac{\partial \left(-\frac{1}{2\rho} \phi^2\right)}{\partial \phi} + \frac{\partial \left(\frac{\rho}{2} (max(0, \ x + \rho^{-1} \phi))^2\right)}{\partial \phi} \\
    & = \quad -\frac{2}{2\rho} \phi + \frac{2 \rho}{2 \rho} (max(0, \ x +\rho^{-1} \phi)) \\
    & = \quad -\frac{\phi}{\rho} + max(0, \ x + \rho^{-1} \phi) \\
    & = \quad max(-\rho^{-1} \phi, \ x)
\end{aligned}


# Exercise 4.3

From *Exercise 4.2* we know that $\frac{\partial g(x, \phi)}{\partial x} = \rho \ max(0, \ x + \rho^{-1} \phi)$ and that $\frac{\partial g(x, \phi)}{\partial \phi} = max(-\rho^{-1} \phi, \ x)$. 

To categorize the function $x \rightarrow g(x, \phi)$ as concave or convexe, all we need to do is verify the signal of $\frac{\partial^2 g(x, \phi)}{\partial x^2}$:

<br>

\begin{aligned}
    \frac{\partial^2 g(x, \phi)}{\partial x^2} \quad & = \quad \frac{\partial (\rho \ max(0, \ x + \rho^{-1} \phi))}{\partial x} \\
    & = \quad \rho \ max(0, 1) \\
    & = \quad \rho
\end{aligned}

<br>

Since by definition we have that $\rho > 0$, we confirm that the function $x \rightarrow g(x, \phi)$ is convex.

Similarly, for the function $\phi \rightarrow g(x, \phi)$, we analyse the signal of $\frac{\partial^2 g(x, \phi)}{\partial \phi^2}$:

<br>

\begin{aligned}
    \frac{\partial^2 g(x, \phi)}{\partial \phi^2} \quad & = \quad \frac{\partial (max(-\rho^{-1} \phi, \ x))}{\partial \phi} \\
    & = \quad \ max(-\rho^{-1}, 0) \\
    & = \quad 0
\end{aligned}

<br>

Since the Hessian of $\phi \rightarrow g(x, \phi)$ is $\leq 0$, the function is concave.

# Exercise 4.4

In [ ]:
def line_search_gradient():
    pass

# Exercise 4.5

In [ ]:
def lagrangian_gradient():
    pass

# Exercise 4.6

In [ ]:
def augmented_lagrangian():
    pass

# Exercise 5.1